In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import nltk
import re
import string
import seaborn as sns
import re
 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
# wget.download("https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/sentiment140-subset.csv.zip")
# !unzip -n sentiment140-subset.csv.zip
 
#loading dataset
data = pd.read_csv('sentiment140-subset.csv', nrows=50000)
print(data.head())

   polarity                                               text
0         0                      @kconsidder You never tweet  
1         0                 Sick today  coding from the couch.
2         1  @ChargerJenn Thx for answering so quick,I was ...
3         1  Wii fit says I've lost 10 pounds since last ti...
4         0  @MrKinetik Not a thing!!!  I don't really have...


In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=400, split=' ')
tokenizer.fit_on_texts(data['text'].values)
 
# Text Vectorization
X = tokenizer.texts_to_sequences(data['text'].values)
X = tf.keras.preprocessing.sequence.pad_sequences(X)
print(X.shape)

(50000, 31)


In [7]:
tokenizer.sequences_to_texts([list(X[0])])

['you never tweet']

In [8]:
embed_dim = 256
lstm_out = 196
max_features = tokenizer.num_words
 
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max_features,
                                    embed_dim, 
                                    input_length = X.shape[1]))
model.add(tf.keras.layers.SpatialDropout1D(0.4))
model.add(tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(lstm_out,
                         dropout=0.05,
                         recurrent_dropout=0.2)))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy',
              optimizer='adam', 
              metrics = ['accuracy'])
print(model.summary())

c:\Users\Windows 11\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
# Create a one-hot encoded representation of the output labels using the get_dummies() method.
Y = pd.get_dummies(data['polarity'])
 
# Map the resultant 0 and 1 values with ‘Positive’ and ‘Negative’ respectively.
result_dict = {0: 'Negative', 1: 'Positive'}
y_arr = np.vectorize(result_dict.get)(Y.columns)
 
Y = Y.values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(33500, 31) (33500, 2)
(16500, 31) (16500, 2)


In [10]:
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 45s 37ms/step - accuracy: 0.6733 - loss: 0.5961
Epoch 2/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 42s 40ms/step - accuracy: 0.7376 - loss: 0.5260
Epoch 3/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 42s 40ms/step - accuracy: 0.7495 - loss: 0.5034
Epoch 4/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 42s 40ms/step - accuracy: 0.7542 - loss: 0.4991
Epoch 5/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.7568 - loss: 0.4909
Epoch 6/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.7619 - loss: 0.4844
Epoch 7/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.7681 - loss: 0.4714
Epoch 8/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.7734 - loss: 0.4616
Epoch 9/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.7816 - loss: 0.4509
Epoch 10/10
1047/1047 ━━━━━━━━━━━━━━━━━━━━ 40s 38ms/step - accuracy: 0.7894 - loss: 0.4395


In [11]:
#First, initialize it.
twt = ['I will recommend this product']
#Next, tokenize it.
twt = tokenizer.texts_to_sequences(twt)
#Pad it
twt = tf.keras.preprocessing.sequence.pad_sequences(twt, maxlen=X.shape[1], dtype='int32', value=0)
#Predict the sentiment by passing the sentence to the model we built.
sentiment = model.predict(twt, batch_size=1)[0]
print(sentiment)
if(np.argmax(sentiment) == 0):
    print(y_arr[0])
elif (np.argmax(sentiment) == 1):
    print(y_arr[1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
[0.19154479 0.8084552 ]
Positive
